# ASP.NET Benchmark Analysis 

This notebook highlights the steps associated with analyzing data from the ASP.NET benchmarks obtained using crank.

In [ ]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent, 3.0.1"
#r "nuget: YamlDotnet"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis, 0.19.1"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Playwright, 1.16.0"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using System.Diagnostics;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

## Building and Using The GC Analysis API

In [ ]:
dotnet build -c Debug "..\GC.Analysis.API"

In [ ]:
#r "C:\performance\artifacts\bin\GC.Infrastructure\Debug\net7.0\GC.Analysis.API.dll"

using GC.Analysis.API;

## Data Acquisition

The next few cells detail how to retrieve the data from a base path. The run name below is the name of the folder generated from running the ``aspnetbenchmarks`` command from the GC.Infrastructure API. 

In [ ]:
// The LoadInfo class consists of all the pertinent fields needed to represent both the result from a particular benchmark
// as well as the the comparison between two runs where the Data2 represents the GCProcessData of the comparand.
public sealed class LoadInfo
{
    public double WorkingSetMB {get;set;} = double.NaN;
    public double PrivateMemoryMB {get;set;} = double.NaN;
    public double Latency50thMS {get; set;} = double.NaN;
    public double Latency75thMS {get; set;} = double.NaN;
    public double Latency90thMS {get; set;} = double.NaN;
    public double Latency99thMS {get; set;} = double.NaN;
    public double MeanLatencyMS {get; set;} = double.NaN;
    public int ProcessId {get;set;}
    public double RequestsPerMSec {get; set;} = double.NaN;
    public string Run {get; set;}
    public GCProcessData Data {get;set;}
    public GCProcessData? Data2 {get;set;}
    public string CommandLine {get;set;}
    public int NumberOfHeapCountSwitches {get;set;} = 0;
    public string Benchmark {get; set;}
    public string Id {get; set;}
    public double TotalSuspensionTimeMSec {get;set;} = double.NaN;
    public double PercentPauseTimeInGC {get; set;} = double.NaN;
    public double PercentTimeInGC {get; set;} = double.NaN;
    public double MeanHeapSizeBeforeMB {get; set;} = double.NaN;
    public double MaxHeapSizeMB {get; set;} = double.NaN;
    public double TotalAllocationsMB {get;set;} = double.NaN;
    public string TracePath {get; set;}
    public string ProcessName {get;set;}
}

In [ ]:
// The DataManager is responsible for parsing all the data from ASP.NET results from a basepath.
public class DataManager 
{
    private readonly Dictionary<string, Dictionary<string, LoadInfo>> _benchmarkToRunData = new();
    private readonly Dictionary<string, Dictionary<string, LoadInfo>> _runToBenchmarkData = new();
    private readonly Dictionary<string, LoadInfo> _data; 
    private readonly string _basePath;

    public DataManager(string basePath)
    {
        _basePath = basePath;
        _data = GetLoadInfoFromBasePath(basePath);
        foreach (var d in _data)
        {
            if (!_benchmarkToRunData.TryGetValue(d.Value.Benchmark, out var runData))
            {
                _benchmarkToRunData[d.Value.Benchmark] = runData = new();
            }
            runData[d.Value.Run] = d.Value;

            if (!_runToBenchmarkData.TryGetValue(d.Value.Run, out var benchmarkData))
            {
                _runToBenchmarkData[d.Value.Run] = benchmarkData = new();
            }

            benchmarkData[d.Value.Benchmark] = d.Value;
        }
    }

    public static double DeltaPercent (double baseline, double comparand) => (comparand - baseline) / baseline * 100;

    public LoadInfo GetComparison(LoadInfo baseline, LoadInfo comparand)
    {
        return new LoadInfo
        {
            WorkingSetMB    = DeltaPercent(baseline.WorkingSetMB, comparand.WorkingSetMB),
            PrivateMemoryMB = DeltaPercent(baseline.PrivateMemoryMB, comparand.PrivateMemoryMB),
            Latency50thMS   = DeltaPercent(baseline.Latency50thMS, comparand.Latency50thMS),
            Latency75thMS   = DeltaPercent(baseline.Latency75thMS, comparand.Latency75thMS),
            Latency90thMS   = DeltaPercent(baseline.Latency90thMS, comparand.Latency90thMS), 
            Latency99thMS   = DeltaPercent(baseline.Latency99thMS, comparand.Latency99thMS),  
            MeanLatencyMS   = DeltaPercent(baseline.MeanLatencyMS, comparand.MeanLatencyMS),
            RequestsPerMSec = DeltaPercent(baseline.RequestsPerMSec, comparand.RequestsPerMSec),
            Data = baseline.Data,
            Data2 = comparand.Data,
            Run = $"{baseline.Run} vs. {comparand.Run}",
            Benchmark = baseline.Benchmark,
            Id = $"{baseline.Run} vs. {comparand.Run} for {baseline.Benchmark}"
        };
    }

    public Dictionary<string, LoadInfo>? GetAllBenchmarksForRun(string run)
    {
        if (!_runToBenchmarkData.TryGetValue(run, out var benchmarksForRun))
        {
            Console.WriteLine($"No benchmarks found for run: {run}");
            return null;
        }

        return benchmarksForRun;
    }

    public void SaveBenchmarkData(string outputPath = "")
    {
        if (string.IsNullOrEmpty(outputPath))
        {
            outputPath = _basePath;
        }

        StringBuilder sb = new();
        sb.AppendLine($"Run,Benchmark,Working Set (MB), Private Memory (MB), Request/MSec, Mean Latency (MSec), Latency 50th Percentile MSec, Latency 75th Percentile MSec, Latency 90th Percentile MSec, Latency 99th Percentile MSec");
        foreach (var b in _data)
        {
            var val = b.Value; 
            sb.AppendLine($"{val.Run},{val.Benchmark},{val.WorkingSetMB},{val.PrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS}");
        }

        File.WriteAllText(Path.Combine(outputPath, "AllBenchmarks.csv"), sb.ToString());
    }

    public Dictionary<string, LoadInfo>? GetAllRunsForBenchmark(string benchmark)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runsForBenchmark))
        {
            Console.WriteLine($"No runs found for benchmark: {benchmark}");
            return null;
        }

        return runsForBenchmark;
    }

    public LoadInfo? GetBenchmarkData(string benchmark, string run)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runData))
        {
            Console.WriteLine($"Benchmark: {benchmark} not found!");
            return null;
        }

        if (!runData.TryGetValue(run, out var loadInfo))
        {
            Console.WriteLine($"Run: {run} not found!");
            return null;
        }

        return loadInfo;
    }

    public Dictionary<string, LoadInfo> Data => _data; 

    private Dictionary<string, LoadInfo> GetLoadInfoFromBasePath(string basePath)
    {
        Dictionary<string, LoadInfo> flatLoadMap = new();
        var files = Directory.GetFiles(basePath, "*.log", SearchOption.AllDirectories);

        foreach (var f in files)
        {
            if (f.Contains("build.log") || f.Contains("output.log") || f.Contains("_GCLog"))
            {
                continue;
            }

            LoadInfo info = new();

            string[] lines = File.ReadAllLines(f);
            int idxOfApplication = Int32.MaxValue;
            int idxOfLoad = Int32.MaxValue;
            int idx = 0;

            foreach (var line in lines)
            {
                string[] sp = line.Split("|", StringSplitOptions.TrimEntries);
                if (line.Contains("| application"))
                {
                    idxOfApplication = idx;
                }

                else if (line.Contains("| load"))
                {
                    idxOfLoad = idx;
                }

                else if (line.Contains("| Latency 50th"))
                {
                    info.Latency50thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 75th"))
                {
                    info.Latency75thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 90th"))
                {
                    info.Latency90thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 99th"))
                {
                    info.Latency99thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("Requests/sec"))
                {
                    info.RequestsPerMSec = double.Parse(sp[2]) / 1000;
                }

                else if (line.Contains("Mean latency"))
                {
                    info.MeanLatencyMS = double.Parse(sp[2]);
                }

                else if (line.Contains("Private Memory") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.PrivateMemoryMB  = double.Parse(sp[2]);
                }

                else if (line.Contains("Working Set") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.WorkingSetMB = double.Parse(sp[2]);
                }

                ++idx;
            }

            string[] split =  f.Replace(".5", "5").Split(".");
            string run = split[1];
            string benchmark = Path.GetFileName( split[0] ).Replace("_Windows", "").Replace("_Linux", "").Replace(".gc", "").Replace(".nettrace", "");

            string key = $"{run} | {benchmark}";
            info.Benchmark = benchmark;
            info.Run = run;
            info.Id = key;

            flatLoadMap[key] = info;
        }

        var traceFiles = Directory.GetFiles(basePath, "*.etl.zip", SearchOption.AllDirectories).ToList();
        var nettraceFiles = Directory.GetFiles(basePath, "*.nettrace", SearchOption.AllDirectories);
        traceFiles.AddRange(nettraceFiles);

        HashSet<string> pertinentProcesses = new HashSet<string>
        {
            "PlatformBenchmarks",
            "Benchmarks",
            "MapAction",
            "TodosApi",
            "BasicGrpc",
            "BasicMinimalApi",
        };

        Parallel.ForEach(traceFiles, (t) => {
            string[] sp = t.Split("\\");
            string benchmark = Path.GetFileNameWithoutExtension(sp[sp.Length - 1]).Replace("_Windows", "").Replace(".gc.etl", "").Replace("_Linux", "").Replace(".nettrace", "").Replace(".gc", "");
            string name = sp[sp.Length - 2].Replace(".5", "5");
            string key = $"{name} | {benchmark}";

            Analyzer analyzer = AnalyzerManager.GetAnalyzer(t);
            GCProcessData? data = null;

            if (t.Contains(".nettrace"))
            {
                data = analyzer.AllGCProcessData.First().Value.First();
            }

            else
            {
                foreach (var p in pertinentProcesses)
                {
                    data = analyzer.GetProcessGCData(p).FirstOrDefault();
                    if (data != null)
                    {
                        break;
                    }
                }
            }

            if (data == null)
            {
                Console.WriteLine($"The following key doesn't have the pertinent process {key} - {t}: {string.Join(" , ", analyzer.TraceLog.Processes.Select(p => p.Name))}");
            }

            else
            {
                lock (flatLoadMap)
                {
                    if (flatLoadMap.TryGetValue(key, out var f))
                    {
                        f.MeanHeapSizeBeforeMB = data.Stats.MeanSizePeakMB;
                        f.MaxHeapSizeMB = data.Stats.MaxSizePeakMB;
                        f.PercentTimeInGC = (data.GCs.Sum(gc => gc.PauseDurationMSec - gc.SuspendDurationMSec) / (data.Stats.ProcessDuration) ) * 100;
                        f.TracePath = data.Parent.TraceLogPath;
                        f.TotalAllocationsMB = data.Stats.TotalAllocatedMB;
                        f.CommandLine = data.CommandLine;
                        f.PercentPauseTimeInGC = data.Stats.GetGCPauseTimePercentage();
                        f.ProcessId = data.ProcessID;
                        f.Data = data;
                        f.ProcessName = data.ProcessName;
                        f.TotalSuspensionTimeMSec = data.GCs.Sum(gc => gc.SuspendDurationMSec);

                        for (int i = 0; i < data.GCs.Count - 1; i++)
                        {
                            if ( data.GCs[i].GlobalHeapHistory?.NumHeaps != data.GCs[i + 1].GlobalHeapHistory?.NumHeaps)
                            {
                                ++f.NumberOfHeapCountSwitches;
                            }
                        }

                    }

                    else
                    {
                        Console.WriteLine($"{key} not found - Check if the trace has any elements: {t}");
                    }
                }
            }
        });

        return flatLoadMap;
    }

    public Dictionary<string, LoadInfo> GetBenchmarkToComparison(string baselineRun, string comparandRun)
    {
        Dictionary<string, LoadInfo> comparisons = new();

        Dictionary<string, LoadInfo> baselineData = new();
        Dictionary<string, LoadInfo> comparandData = new();
        HashSet<string> allBenchmarks = new();

        foreach (var d in _data)
        {
            allBenchmarks.Add(d.Value.Benchmark);

            string run = d.Key.Split("|", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries)[0];

            if (string.CompareOrdinal(run, baselineRun) == 0 && !baselineData.TryGetValue(d.Key, out var baselineInfo))
            {
                baselineInfo = baselineData[d.Value.Benchmark] = d.Value;
            }

            else if (string.CompareOrdinal(run, comparandRun) == 0 && !comparandData.TryGetValue(d.Key, out var comparandInfo))
            {
                comparandInfo = comparandData[d.Value.Benchmark] = d.Value;
            }
        }

        foreach (var benchmark in allBenchmarks)
        {
            if (!baselineData.TryGetValue(benchmark, out var baselineBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the baseline: {baselineRun}");
                continue;
            }

            if (!comparandData.TryGetValue(benchmark, out var comparandBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the comparand: {comparandRun}");
                continue;
            }

            LoadInfo comparison = GetComparison(baselineBenchmarkInfo, comparandBenchmarkInfo);
            comparisons[benchmark] = comparison;
        }
        
        return comparisons;
    }
}

In [ ]:
string basePath = @"Enter your basepath containing the ASP.NET artifacts here.";
var dataManager = new DataManager(basePath);

In [ ]:
dataManager.Data

### Using the DataManager

The following cells demonstrates how to make use of the ``DataManager``. 

In [ ]:
// The name of the run from the yaml file for which the ASP.NET run is created for.
string runName = "run";

Dictionary<string, LoadInfo> run = dataManager.GetAllBenchmarksForRun(runName);
dataManager.Data.Display();
List<KeyValuePair<string, LoadInfo>> runsWithGCData = dataManager.GetAllBenchmarksForRun(runName).Where(gc => gc.Value.Data != null);

In [ ]:
string benchmarkName = "Name of the specific benchmark";
LoadInfo benchmarkData = dataManager.GetBenchmarkData(benchmark: benchmarkName, run: runName);
benchmarkData.Id

In [ ]:
Dictionary<string, LoadInfo> allRunsForBenchmark = dataManager.GetAllRunsForBenchmark(benchmark: benchmarkName);
allRunsForBenchmark.Keys

#### Saving The Benchmark Results

The following call will persist a flat list of all the results.

In [ ]:
dataManager.SaveBenchmarkData("./")

## Build to Build Comparison and Volatility Analysis

In [ ]:
var run1_vs_run2 = dataManager.GetBenchmarkToComparison("datas_2", "datas_3");

In [ ]:
static bool IsNotInvalidDouble(double val) => 
    !double.IsNaN(val) && 
    !double.IsInfinity(val) && 
    !double.IsPositiveInfinity(val) && 
    !double.IsNegativeInfinity(val);

public class SummaryTable
{
    public SummaryTable(Dictionary<string, Dictionary<string, LoadInfo>> comparisons)
    {
        Comparisons = comparisons;
    }

    private string GenerateSummaryForComparison(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        double averageWorkingSet = comparison.Where(a => IsNotInvalidDouble(a.Value.WorkingSetMB)).Average(a => a.Value.WorkingSetMB);
        double privateMemory = comparison.Where(a => IsNotInvalidDouble(a.Value.PrivateMemoryMB)).Average(a => a.Value.PrivateMemoryMB);
        double throughput = comparison.Where(a => IsNotInvalidDouble(a.Value.RequestsPerMSec)).Average(a => a.Value.RequestsPerMSec);
        double meanLatency = comparison.Where(a => IsNotInvalidDouble(a.Value.MeanLatencyMS)).Average(a => a.Value.MeanLatencyMS);

        double p50Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency50thMS)).Average(a => a.Value.Latency50thMS);
        double p75Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency75thMS)).Average(a => a.Value.Latency75thMS);
        double p90Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency90thMS)).Average(a => a.Value.Latency90thMS);
        double p99Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency99thMS)).Average(a => a.Value.Latency99thMS);

        return $"{comparisonKey},{averageWorkingSet},{privateMemory},{throughput},{meanLatency},{p50Latency},{p75Latency},{p90Latency},{p99Latency}";
    }

    public string GenerateSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Average Working Set (MB) %, Average Private Memory (MB) %, Average Request/MSec %, Average Mean Latency (MSec), Average P50 Latency (MSec) %, Average P75 Latency (MSec) %, Average P90 Latency (MSec) %, Average P99 Latency (MSec) %");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateSummaryForComparison(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    private int GetCountOfRegressions(List<double> selected, double thresholdPercentage, bool lessIsBetter = true)
    {
        // If throughput, less is worse => threshold <= -5%.
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && ( (lessIsBetter) ? (d >= thresholdPercentage) : (d <= -thresholdPercentage)));
        return comparison.Count;
    }

    private int GetCountOfAbsRegressions(List<double> selected, double thresholdPercentage)
    {
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && Math.Abs(d) >= thresholdPercentage);
        return comparison.Count;
    }

    // # of benchmarks with throughput regressed by >= 5% and 10%
    private string GenerateRegressionSummary(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        List<double> workingSet  = comparison.Select(c => c.Value.WorkingSetMB);
        int workingSetCountGT_5  = GetCountOfRegressions(workingSet, 5);
        int workingSetCountGT_10 = GetCountOfRegressions(workingSet, 10);

        List<double> privateMemory  = comparison.Select(c => c.Value.PrivateMemoryMB);
        int privateMemoryCountGT_5  = GetCountOfRegressions(privateMemory, 5);
        int privateMemoryCountGT_10 = GetCountOfRegressions(privateMemory, 10);

        List<double> throughput  = comparison.Select(a => a.Value.RequestsPerMSec);
        int throughputCountGT_5  = GetCountOfRegressions(throughput, 5, false);
        int throughputCountGT_10 = GetCountOfRegressions(throughput, 10, false);

        List<double> meanLatency  = comparison.Select(a => a.Value.MeanLatencyMS);
        int meanLatencyCountGT_5  = GetCountOfRegressions(meanLatency, 5);
        int meanLatencyCountGT_10 = GetCountOfRegressions(meanLatency, 10);

        List<double> p50Latency  = comparison.Select(a => a.Value.Latency50thMS);
        int p50LatencyCountGT_5  = GetCountOfRegressions(p50Latency, 5);
        int p50LatencyCountGT_10 = GetCountOfRegressions(p50Latency, 10);

        List<double> p75Latency  = comparison.Select(a => a.Value.Latency75thMS);
        int p75LatencyCountGT_5  = GetCountOfRegressions(p75Latency, 5);
        int p75LatencyCountGT_10 = GetCountOfRegressions(p75Latency, 10);

        List<double> p90Latency  = comparison.Select(a => a.Value.Latency90thMS);
        int p90LatencyCountGT_5  = GetCountOfRegressions(p90Latency, 5);
        int p90LatencyCountGT_10 = GetCountOfRegressions(p90Latency, 10);
        
        List<double> p99Latency  = comparison.Select(a => a.Value.Latency99thMS);
        int p99LatencyCountGT_5  = GetCountOfRegressions(p99Latency, 5);
        int p99LatencyCountGT_10 = GetCountOfRegressions(p99Latency, 10);

        return $"{comparisonKey},{workingSetCountGT_5},{workingSetCountGT_10},{privateMemoryCountGT_5},{privateMemoryCountGT_10},{throughputCountGT_5},{throughputCountGT_10},{meanLatencyCountGT_5},{meanLatencyCountGT_10},{p50LatencyCountGT_5},{p50LatencyCountGT_10},{p75LatencyCountGT_5},{p75LatencyCountGT_10},{p90LatencyCountGT_5},{p90LatencyCountGT_10},{p99LatencyCountGT_5},{p99LatencyCountGT_10}";
    }

    public string GenerateRegressionSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Reg. Count - Working Set (MB),Large Reg. Count - Working Set (MB),Reg. Count - Private Memory (MB),Large Reg. Count - Private Memory (MB),Reg. Count - Throughput, Large Reg. Count - Throughput,Reg. Count - Mean Latency,Large Reg. Count - Mean Latency,Reg. Count - P50 Latency, Large Reg. Count - P50 Latency, Reg. Count - P75 Latency, Large Reg. Count - P75 Latency,Reg. Count - P90 Latency, Large Reg. Count - P90 Latency,Reg. Count - P99 Latency, Large Reg. Count - P99 Latency");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateRegressionSummary(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    public Dictionary<string, string> GenerateRegressionAnalysisForComparison(string comparisonKey)
    {
        StringBuilder sb = new();
        Dictionary<string, string> csvData = new();
        Dictionary<string, LoadInfo> comparison = Comparisons[comparisonKey];

        string header = "Benchmark,WorkingSetMB,PrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS";

        // Generate Memory Regressions.
        StringBuilder memRegressions = new();
        memRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.WorkingSetMB >= 10 || c.Value.PrivateMemoryMB >= 10 ))
        {
            memRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.WorkingSetMB},{benchmark.Value.PrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["memory"] = memRegressions.ToString();

        // Generate Throughput Regressions.
        StringBuilder throughputRegressions = new();
        throughputRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.RequestsPerMSec <= -10))
        {
            throughputRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.WorkingSetMB},{benchmark.Value.PrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["throughput"] = throughputRegressions.ToString();

        // Generate Latency Regressions.
        StringBuilder latencyRegressions = new();
        latencyRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MeanLatencyMS >= 10 || 
                                                        c.Value.Latency50thMS >= 10 || 
                                                        c.Value.Latency75thMS >= 10 || 
                                                        c.Value.Latency90thMS >= 10 || 
                                                        c.Value.Latency99thMS >= 10 ))
        {
            latencyRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.WorkingSetMB},{benchmark.Value.PrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["latency"] = latencyRegressions.ToString();

        // All.
        StringBuilder all = new();
        all.AppendLine(header);
        foreach (var benchmark in comparison)
        {
            all.AppendLine($"{benchmark.Key},{benchmark.Value.WorkingSetMB},{benchmark.Value.PrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["all"] = all.ToString();

        return csvData;
    }

    public void SaveComparisons(string basePath)
    {
        // Add Summary for Comparisons.
        string summaryOfComparisons = GenerateSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "SummaryOfComparisons.csv"), summaryOfComparisons);

        // Add Regression Summary for Comparisons.
        string regressionSummary = GenerateRegressionSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "RegressionSummary.csv"), regressionSummary);

        // Add Large Regression Analysis for Comparison.
        string perComparisonDataPath = Path.Combine(basePath, "PerComparisonData");
        if (!Directory.Exists(perComparisonDataPath))
        {
            Directory.CreateDirectory(perComparisonDataPath);
        }

        foreach (var comparison in Comparisons)
        {
            string comparisonPath = Path.Combine(perComparisonDataPath, comparison.Key);
            Directory.CreateDirectory(comparisonPath);

            Dictionary<string, string> regressionComparisons = GenerateRegressionAnalysisForComparison(comparison.Key);

            // Memory
            File.WriteAllText(Path.Combine(comparisonPath, "MemoryRegressions.csv"), regressionComparisons["memory"]);

            // Throughput
            File.WriteAllText(Path.Combine(comparisonPath, "ThroughputRegressions.csv"), regressionComparisons["throughput"]);

            // Latency
            File.WriteAllText(Path.Combine(comparisonPath, "LatencyRegressions.csv"), regressionComparisons["latency"]);

            // All
            File.WriteAllText(Path.Combine(comparisonPath, "All.csv"), regressionComparisons["all"]);
        }
    }

    public Dictionary<string, Dictionary<string, LoadInfo>> Comparisons { get; }
}

In [ ]:
Dictionary<string, Dictionary<string, LoadInfo>> comparisons = new()
{
    { nameof(run1_vs_run2), run1_vs_run2 },
};

SummaryTable summaryTable = new(comparisons);
summaryTable.SaveComparisons("./");

## Charting Helpers

The following cells highlight how to chart certain properties of the LoadInfo class.

In [ ]:
void ChartProperty(LoadInfo baseline, LoadInfo comparand, string nameOfProperty)
{
    GCProcessData baselineGC = baseline.Data;
    GCProcessData comparandGC = comparand.Data;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for {baseline.Id}" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for {comparand.Id}" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison Between {baseline.Run} and {comparand.Run}", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(LoadInfo comparison, string nameOfProperty)
{
    GCProcessData baselineGC = comparison.Data;
    GCProcessData comparandGC = comparison.Data2;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for Baseline" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for Comparand" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

In [ ]:
dataManager.Data

In [ ]:
var run1_Benchmark = dataManager.GetBenchmarkData(benchmark: "CachingPlatform", "datas_2");
var run2_Benchmark = dataManager.GetBenchmarkData(benchmark: "CachingPlatform", "datas_3");

// Chart the PauseDurationMSec for the run1 vs. run2.
ChartProperty(baseline: run1_Benchmark, comparand: run2_Benchmark, nameof(TraceGC.PauseDurationMSec))

## Debugging

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id

In [ ]:
#!about